# Logistic Regression #2: Multi-Sauce Recommendation System

## Objective
Build a recommendation system that predicts which sauce(s) a customer is likely to purchase based on their cart contents.

We train **one model per sauce** and compare their performance.

### Available Sauces:
- Crazy Sauce
- Cheddar Sauce
- Extra Cheddar Sauce
- Garlic Sauce
- Tomato Sauce
- Blueberry Sauce
- Spicy Sauce
- Pink Sauce

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from data_loader import load_raw_data, SAUCES
from preprocessing import prepare_lr_data, normalize_features, get_feature_names
from models.logistic_regression import LogisticRegression
from models.evaluation import (
    accuracy, precision, recall, f1_score, 
    confusion_matrix, classification_report
)

# Settings
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print("Available sauces:")
for sauce in SAUCES:
    print(f"  - {sauce}")

## 1. Load Data

In [ ]:
# Load raw data
DATA_PATH = Path('../data/raw/ap_dataset.csv')
df = load_raw_data(DATA_PATH)

print(f"Dataset: {len(df)} rows, {df['id_bon'].nunique()} receipts")

In [ ]:
# Check sauce distribution
sauce_counts = {}
total_receipts = df['id_bon'].nunique()

for sauce in SAUCES:
    count = df[df['retail_product_name'] == sauce]['id_bon'].nunique()
    sauce_counts[sauce] = {
        'count': count,
        'percentage': 100 * count / total_receipts
    }

sauce_df = pd.DataFrame(sauce_counts).T
sauce_df = sauce_df.sort_values('count', ascending=False)
print("Sauce Purchase Rates:")
sauce_df

## 2. Train Models for Each Sauce

In [ ]:
from sklearn.model_selection import train_test_split

# Store results
models = {}
results = {}

for sauce in SAUCES:
    print(f"\n{'='*60}")
    print(f"Training model for: {sauce}")
    print('='*60)
    
    # Prepare data (no filter - all receipts)
    try:
        X, y, features_df = prepare_lr_data(
            df,
            target_sauce=sauce,
            filter_product=None,  # All receipts
            exclude_all_sauces=True
        )
    except Exception as e:
        print(f"  Error preparing data: {e}")
        continue
    
    # Check if we have enough positive samples
    if y.sum() < 10:
        print(f"  Skipping - not enough positive samples ({y.sum()})")
        continue
    
    print(f"  Samples: {len(y)}, Positive: {y.sum()} ({100*y.mean():.1f}%)")
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Normalize
    X_train_norm, mean, std = normalize_features(X_train)
    X_test_norm, _, _ = normalize_features(X_test, mean=mean, std=std)
    
    # Train model
    model = LogisticRegression(
        learning_rate=0.1,
        n_iterations=500,
        regularization=0.01,
        verbose=False
    )
    model.fit(X_train_norm, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test_norm)
    
    report = classification_report(y_test, y_pred)
    
    print(f"  Accuracy:  {report['accuracy']:.4f}")
    print(f"  Precision: {report['precision']:.4f}")
    print(f"  Recall:    {report['recall']:.4f}")
    print(f"  F1 Score:  {report['f1_score']:.4f}")
    
    # Store
    models[sauce] = {
        'model': model,
        'mean': mean,
        'std': std,
        'feature_names': get_feature_names(features_df)
    }
    results[sauce] = {
        'samples': len(y),
        'positive_rate': y.mean(),
        **report
    }

## 3. Compare Model Performance

In [ ]:
# Create comparison DataFrame
comparison_data = []
for sauce, res in results.items():
    comparison_data.append({
        'Sauce': sauce,
        'Base Rate': res['positive_rate'],
        'Accuracy': res['accuracy'],
        'Precision': res['precision'],
        'Recall': res['recall'],
        'F1 Score': res['f1_score'] 
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('F1 Score', ascending=False)

print("Model Performance Comparison:")
comparison_df.round(4)

In [ ]:
# Visualization - metrics comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
colors = ['steelblue', 'seagreen', 'coral', 'mediumpurple']

for ax, metric, color in zip(axes.flatten(), metrics, colors):
    data = comparison_df.sort_values(metric, ascending=True)
    ax.barh(data['Sauce'], data[metric], color=color)
    ax.set_xlabel(metric)
    ax.set_title(f'{metric} by Sauce')
    ax.set_xlim(0, 1)
    
plt.tight_layout()
plt.savefig('../results/lr2_metrics_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Heatmap of all metrics
fig, ax = plt.subplots(figsize=(10, 8))

heatmap_data = comparison_df.set_index('Sauce')[['Base Rate', 'Accuracy', 'Precision', 'Recall', 'F1 Score']]
sns.heatmap(heatmap_data, annot=True, fmt='.3f', cmap='YlGnBu', ax=ax,
            cbar_kws={'label': 'Score'})
ax.set_title('Multi-Sauce Model Performance Heatmap')
plt.tight_layout()
plt.savefig('../results/lr2_performance_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Feature Importance Analysis

In [ ]:
# Top features for each sauce
print("Top 5 Features for Each Sauce Model:")
print("=" * 70)

for sauce, model_data in models.items():
    model = model_data['model']
    feature_names = model_data['feature_names']
    
    importance = model.get_feature_importance(feature_names)
    top_5 = list(importance.items())[:5]
    
    print(f"\n{sauce}:")
    for name, data in top_5:
        direction = "+" if data['weight'] > 0 else "-"
        print(f"  {direction} {name}: {data['weight']:+.4f}")

In [ ]:
# Feature importance heatmap for top features
# Get union of top 10 features across all models
all_top_features = set()
for sauce, model_data in models.items():
    importance = model_data['model'].get_feature_importance(model_data['feature_names'])
    all_top_features.update(list(importance.keys())[:10])

all_top_features = list(all_top_features)

# Create weight matrix
weight_matrix = pd.DataFrame(index=all_top_features, columns=list(models.keys()))

for sauce, model_data in models.items():
    importance = model_data['model'].get_feature_importance(model_data['feature_names'])
    for feat in all_top_features:
        if feat in importance:
            weight_matrix.loc[feat, sauce] = importance[feat]['weight']
        else:
            weight_matrix.loc[feat, sauce] = 0

weight_matrix = weight_matrix.astype(float)

In [ ]:
# Visualize feature weights across sauces
fig, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(weight_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0, ax=ax,
            cbar_kws={'label': 'Weight'})
ax.set_title('Feature Weights Across Sauce Models\n(Red = positive, Blue = negative)')
ax.set_xlabel('Sauce')
ax.set_ylabel('Feature')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../results/lr2_feature_weights_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Recommendation Function

In [ ]:
def recommend_sauces(cart_features, models, top_n=3):
    """
    Recommend top N sauces based on cart features.
    
    Args:
        cart_features: Dictionary of feature_name -> value
        models: Dictionary of sauce -> model info
        top_n: Number of sauces to recommend
        
    Returns:
        List of (sauce_name, probability) tuples
    """
    predictions = []
    
    for sauce, model_data in models.items():
        model = model_data['model']
        mean = model_data['mean']
        std = model_data['std']
        feature_names = model_data['feature_names']
        
        # Create feature vector
        X = np.zeros(len(feature_names))
        for i, feat in enumerate(feature_names):
            if feat in cart_features:
                X[i] = cart_features[feat]
        
        # Normalize
        X_norm = (X - mean) / std
        
        # Predict
        prob = model.predict_proba(X_norm.reshape(1, -1))[0]
        predictions.append((sauce, prob))
    
    # Sort by probability
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    return predictions[:top_n]

# Example usage
example_cart = {
    'has_Crazy Schnitzel': 1,
    'has_French fries': 1,
    'cart_size': 3,
    'total_value': 50,
    'hour': 13,
    'is_weekend': 0
}

print("Example Recommendation:")
print(f"Cart: {example_cart}")
print("\nRecommended Sauces:")
recommendations = recommend_sauces(example_cart, models, top_n=3)
for sauce, prob in recommendations:
    print(f"  {sauce}: {prob:.1%} probability")

## 6. Summary

In [ ]:
print("=" * 70)
print("MULTI-SAUCE RECOMMENDATION SYSTEM SUMMARY")
print("=" * 70)
print(f"\nModels trained: {len(models)}")
print(f"\nPerformance Summary:")

avg_acc = comparison_df['Accuracy'].mean()
avg_f1 = comparison_df['F1 Score'].mean()
best_sauce = comparison_df.iloc[0]['Sauce']
best_f1 = comparison_df.iloc[0]['F1 Score']

print(f"  Average Accuracy: {avg_acc:.4f}")
print(f"  Average F1 Score: {avg_f1:.4f}")
print(f"  Best Model: {best_sauce} (F1: {best_f1:.4f})")

print(f"\nRecommendation System Ready!")
print(f"Use recommend_sauces(cart_features, models) to get recommendations.")
print("=" * 70)